In [17]:
!pip install tflearn

In [28]:
#he Natural Language Toolkit (NLTK) is a Python package for natural language processing
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [26]:
import numpy as np
import tensorflow as tf
import tflearn
import random
import json
import pickle

In [29]:
try:
  nltk.download('punkt') #package
except:
  pass

with open('intents.json')as file:
  data = json.load(file)
try:
  with open("data.pickle","rb")as f:
    words ,labels ,training,output = pickle.load(f)
except:  
  words = []
  labels = []
  docs_x = []
  docs_y = []

  for intent in data['intents']:
    for pattern in intent['patterns']:
      words2 = nltk.word_tokenize(pattern)
      words.extend(words2)
      docs_x.append(words2)
      docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
      labels.append(intent['tag'])
  #converting each word into lowercase & removing
  words = [stemmer.stem(w.lower()) for w in words if w not in "?"]

  #sorting and removing duplicates
  words = sorted(list(set(words))) 

  labels = sorted(labels)    
  # print(words2)
  # print(docs_y)
#----------------------------------------------------------------#
  # print(docs_x)
  training = []
  output = []

  out_empty = [0 for _ in range(len(labels))]

  for x,doc in enumerate(docs_x):
    bag = []
    words3 = [stemmer.stem(w) for w in doc]
    for w in words:
      if w in words3:
        bag.append(1)
      else:
        bag.append(0)
    outputRow = out_empty[:]
    outputRow [labels.index(docs_y[x])]=1

    training.append(bag)
    output.append(outputRow)

  training = np.array(training)
  output = np.array(output)
  with open("data.pickle","wb")as f:
      pickle.dump((words ,labels ,training,output),f) 


[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


as the neural networks only accepts number and we have a list of words so we need to use something like one-hot encoding, this means that if the word exists we will append one (even if it exists for more than one time)
and if it is not exist we will append 0.

###Model building

In [30]:
tf.compat.v1.reset_default_graph()

net  = tflearn.input_data(shape=[None,len(training[0])])#input layer
net = tflearn.fully_connected(net,8) #hidden layer
net = tflearn.fully_connected(net,8)#hidden layer
net = tflearn.fully_connected(net,8)#hidden layer
net = tflearn.fully_connected(net,8)#hidden layer
net = tflearn.fully_connected(net,len(output[0]),activation='softmax')#output layer
net = tflearn.regression(net)
model = tflearn.DNN(net)
try:
  #model.load("model.tflearn")
  model.py
except:
  model.fit(training,output,n_epoch=1500, batch_size = 16 ,show_metric = True)
  model.save("model.tflearn")

Training Step: 2999  | total loss: 0.09218 | time: 0.005s
| Adam | epoch: 1500 | loss: 0.09218 - acc: 0.9966 -- iter: 16/25
Training Step: 3000  | total loss: 0.08837 | time: 0.010s
| Adam | epoch: 1500 | loss: 0.08837 - acc: 0.9969 -- iter: 25/25
--
INFO:tensorflow:c:\Users\Lenovo\Desktop\Fourth-level\2ndSemester\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [31]:
def bag_of_words(y,words):
  bag=[0 for _ in range(len(words))]
  seen_words=nltk.word_tokenize(y)
  seen_words = [stemmer.stem(word.lower())for word in seen_words]
  for seen in seen_words :
    for i,w in enumerate(words):
      if w == seen :
        bag[i] = 1
  return np.array(bag)      

def chat():
    print("start talking with the bot!")
    while True:
        inputs = input("you :")
        if inputs.lower() == "quit":
            break
        results = model.predict([bag_of_words(inputs, words)])
        results_index = np.argmax(results)
        #print(results_index)
        tag = labels[results_index]
      
        for t in data["intents"]:
            if t['tag'] == tag:
                responses = t['responses']
        print(random.choice(responses))
    
chat()


start talking with the bot!
Hi there, what can I do for you?
you can go to the gym!
Have a nice day
